# Plotting: histograms

In this notebook, we illustrate the possibilities of plotting 1D and 2D histograms.

Note that Osyris's plotting functions are wrapping Matplotlib's plotting functions,
and forwards most Matplotlib arguments to the underlying function.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

path = "osyrisdata/starformation"
data = osyris.Dataset(8, path=path).load()

## 1D histograms

The `histogram1d` function provides a simple way to make 1D histogram plots.
When a vector quantity is supplied, by default `histogram1d` will use the norm of the vectors.

### A 1D histogram of gas density

For example, to plot a histogram of the gas density, simply do

In [ ]:
osyris.histogram1d(data["hydro"]["density"], logx=True)

### Specifying the bins

The bin edges can be specified using the `bins` parameter,
which can either be an integer number or an array (similarly to Numpy's `bins` argument):

In [ ]:
osyris.histogram1d(data["hydro"]["density"], logx=True,
                   bins=np.logspace(-18., -13., 10))

### Weighted histogram

By default, `histogram1d` will show a binned count of cells, but it does also support weights.
For example, creating a mass-weighted histogram of the gas density can be achieved via

In [ ]:
data["hydro"]["mass"] = (data["hydro"]["density"] * (data["amr"]["dx"]**3)).to("M_sun")
osyris.histogram1d(data["hydro"]["density"],
                   weights=data["hydro"]["mass"],
                   logx=True)

### Multiple 1D histograms

Multiple histograms can be over-plotted on the same axes by using multiple layers:

In [ ]:
bins = np.linspace(-0.15, 0.15, 40)
osyris.histogram1d({"data": data["hydro"]["B_field"].x, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].y, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].z, "alpha": 0.5},
                   logy=True, bins=bins)

It is also possible to specify different bins for different layers:

In [ ]:
osyris.histogram1d({"data": data["hydro"]["B_field"].x, "alpha": 0.5, "bins": 40},
                   {"data": data["hydro"]["B_field"].y, "alpha": 0.5, "bins": 10},
                   logy=True)

## 2D histograms

The `histogram2d` function can be used to make 2D histograms with two different quantities as input.
When a vector quantity is supplied, by default `histogram2d` will use the norm of the vectors

### A 2D histogram of gas density vs magnetic field magnitude

To create a 2D histogram of gas density vs magnetic field magnitude, use

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True)

### Changing the colorscale

The colormap and the range of values can be changed as follows. 

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True,
                   cmap="magma", vmin=10., vmax=1000.)

### Controlling the horizontal and vertical range

To control the range covered by the horizontal and vertical binning,
the `xmin`, `xmax`, `ymin`, `ymax` arguments are provided.

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True, xmax=1.0e-16, ymax=1.0e-3)

### Using a layer for the colormap instead of counting cells

By default, `histogram2d` will show a binned count of cells.
However, the colors can represent the histogram of a supplied `Array` instead.

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"],
    {"data": data["hydro"]["mass"], "norm": "log"},
    loglog=True)

### Applying a mean operation inside each bin

By default, the sum of the layer values in computed inside each bin.
It can sometimes be useful to compute the mean inside each bin instead,
and this can be done by setting `operation='mean'`.

For example, we can get a feel for the resolution distribution in our histogram by histogramming the AMR `level` of the cells,
and applying a `'mean'` operation inside the pixels.

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"],
    {"data": data["amr"]["level"], "operation": "mean"},
    loglog=True)

### Multiple layers

One can use any number of layers to overlay, although anything being two layers is probably not very useful.

In [ ]:
osyris.histogram2d(
    data["hydro"]["density"], data["hydro"]["B_field"],
    {"data": data["hydro"]["mass"], "norm": "log"}, # layer 1
    {"data": data["amr"]["level"], "operation": "mean", "fmt": "%i",
     "mode": "contour", "colors": "k", "levels": [5, 6, 7, 8, 9]}, # layer 2
    loglog=True)

### Controlling the resolution

By default, the histograms have a resolution of 256x256 pixels.
This can be changed via the keyword argument `resolution`,
which can either be an integer (in which case the same resolution is applied to both the x and y dimension)
or a dict with the syntax `resolution={'x': nx, 'y': ny}`.

In [ ]:
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True,
                   resolution=64)

## Subplots / tiled plots

Osyris has no built-in support for subplots (also known as tiled plots).
Instead, we leverage Matplotlib's ability to create such layouts.
Osyris plots are then inserted into the Matplotlib axes, using the `ax` argument.

In the example below, we create four panels and insert various histograms.

In [ ]:
# Create figure
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2, 2, figsize=(12, 9))

osyris.histogram1d(data["hydro"]["density"], logx=True, ax=ax[0, 0])
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["B_field"],
                   norm="log", loglog=True, ax=ax[0, 1])
osyris.histogram1d({"data": data["hydro"]["B_field"].x, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].y, "alpha": 0.5},
                   {"data": data["hydro"]["B_field"].z, "alpha": 0.5},
                   logy=True, bins=np.linspace(-0.15, 0.15, 40), ax=ax[1, 0])
osyris.histogram2d(data["hydro"]["density"], data["hydro"]["velocity"],
                   {"data": data["hydro"]["mass"], "norm": "log"},
                   loglog=True, cmap="magma", ax=ax[1, 1])